In [4]:
from __future__ import division, print_function
import numpy as np
import os
import sys
sys.path.append('../hw5/homework5_code')
from hmmlearn.hmm import MultinomialHMM

In [281]:
%run 'preprocessing.ipynb'

In [282]:
def get_rhymes(words):
    num_words = len(words)
    rhymes = np.empty((num_words, num_words))
    
    for i in range(num_words):
        for j in (k for k in range(num_words) if not k == i):
            if poetrytools.rhymes(words[i], words[j]):
                rhymes[i][j] = 1
    return rhymes

In [283]:
quatrain_rhymes = get_rhymes(quatrain_words)
volta_rhymes = get_rhymes(volta_words)
couplet_rhymes = get_rhymes(couplet_words)

In [366]:
reversed_quatrain_model = MultinomialHMM(n_components=30, n_iter=20, verbose=True)
reversed_volta_model = MultinomialHMM(n_components=30, n_iter=20, verbose=True)
reversed_couplet_model = MultinomialHMM(n_components=30, n_iter=20, verbose=True)
reversed_quatrain_model.fit(np.concatenate(reversed_converted_quatrain_lines), quatrain_lengths)
reversed_volta_model.fit(np.concatenate(reversed_converted_volta_lines), volta_lengths)
reversed_couplet_model.fit(np.concatenate(reversed_converted_couplet_lines), couplet_lengths)

         1      -76047.7795             +nan
         2      -62170.0179      +13877.7617
         3      -62164.7930          +5.2249
         4      -62149.2013         +15.5917
         5      -62102.8951         +46.3062
         6      -61975.0485        +127.8466
         7      -61681.2610        +293.7875
         8      -61191.9464        +489.3146
         9      -60640.8714        +551.0749
        10      -60215.1222        +425.7493
        11      -59976.1031        +239.0191
        12      -59849.8987        +126.2044
        13      -59748.0594        +101.8393
        14      -59611.3014        +136.7581
        15      -59408.7728        +202.5286
        16      -59160.3052        +248.4676
        17      -58925.3034        +235.0018
        18      -58732.8572        +192.4462
        19      -58553.4725        +179.3847
        20      -58333.3897        +220.0828
         1      -35793.6126             +nan
         2      -30165.6174       +5627.9952
         3

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=30,
        n_iter=20, params='ste',
        random_state=<mtrand.RandomState object at 0x10dabd640>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=True)

In [367]:
def generate_b_line(seed, rhymes, model, length=7):
    b_line = []
    start_probs = model.startprob_
    emission_probs = model.emissionprob_
    transition_probs = model.transmat_

    start_state = np.random.choice(len(start_probs), p = start_probs)
    
    possible_start_emissions = np.where(rhymes[seed] == 1)
    probs = np.array(emission_probs[start_state][possible_start_emissions])

    scaled_probs = probs / sum(probs)
    
    start_emission = np.random.choice(possible_start_emissions[0], p=scaled_probs)
    b_line.append(start_emission)
    
    curr_state = start_state
    for _ in range(length - 1):
        possible_transitions = transition_probs[curr_state]
        curr_state = np.random.choice(len(possible_transitions), p=possible_transitions)
        possible_emissions = emission_probs[curr_state]
        curr_emission = np.random.choice(len(possible_emissions), p=possible_emissions)
        b_line.append(curr_emission)
        
    return b_line

def convert_a_line(sample, words):
    ret = ''
    reversed_list = []
    for word in np.nditer(sample):
        reversed_list.append(word)
    
    i = 0
    for word in reversed(reversed_list):
        curr_word = words[word]
        if i == 0 or len(curr_word) == 1:
            ret += curr_word.title() + ' '
        else:
            ret += curr_word + ' '
        i += 1
    return ret

def convert_b_line(sample, words):
    ret = ''
    i = 0
    for word in reversed(sample):
        curr_word = words[word]
        if i == 0 or len(curr_word) == 1:
            ret += curr_word.title() + ' '
        else:
            ret += curr_word + ' '
        i += 1
    return ret

def generate_pair(model, rhymes, length=7):
    while True:
        a_line = model.sample(length)[0]
        seed = a_line[0][0]
        if len(np.where(rhymes[seed] == 1)[0]) > 0:
            b_line = generate_b_line(seed, rhymes, model)
            return a_line, b_line
        
def generate_rhyming_sonnet():
    sonnet = ''
    a_lines = []
    b_lines = []
    
    for _ in range(4):
        a_line, b_line = generate_pair(reversed_quatrain_model, quatrain_rhymes)
        a_lines.append(a_line)
        b_lines.append(b_line)
    
    for i in range(2):
        sonnet += convert_a_line(a_lines[2 * i], quatrain_words) + '\n'
        sonnet += convert_a_line(a_lines[2 * i + 1], quatrain_words) + '\n'
        sonnet += convert_b_line(b_lines[2 * i], quatrain_words) + '\n'
        sonnet += convert_b_line(b_lines[2 * i + 1], quatrain_words) + '\n'
    
    a_lines = []
    b_lines = []
    
    for _ in range(2):
        a_line, b_line = generate_pair(reversed_volta_model, volta_rhymes)
        a_lines.append(a_line)
        b_lines.append(b_line)
    
    sonnet += convert_a_line(a_lines[0], volta_words) + '\n'
    sonnet += convert_a_line(a_lines[1], volta_words) + '\n'
    sonnet += convert_b_line(b_lines[0], volta_words) + '\n'
    sonnet += convert_b_line(b_lines[1], volta_words) + '\n'
    
    a_line, b_line = generate_pair(reversed_couplet_model, couplet_rhymes)
    sonnet += convert_a_line(a_line, couplet_words) + '\n'
    sonnet += convert_b_line(b_line, couplet_words) + '\n'
    
    return sonnet

def generate_10_rhyming_sonnets():
    sonnets = ''
    for i in range(10):
        sonnets += str(i) + '\n' + generate_rhyming_sonnet() + '\n'
    
    f = open("project2data/rhyming_shakespeare.txt","w")
    f.write(sonnets)
    return sonnets

In [368]:
print(generate_10_rhyming_sonnets())

0
For my side high envy altered behold 
Love by doth from hath little befits 
For are alltyrant which to sweetest bold 
Than and me that me despised monuments 
Such and while not A thy die 
Why our that with closet fawn shaken 
Eyes needs my rage no summer dye 
Every dost if after but one reason 
Weighs my how franticmad that the prove 
And that I my self sweet rhyme 
Thine the and knows A vices move 
May due fame my made the prime 
That this and winter evil one beloved 
Twice illused what's with full shine loved 

1
Not you above my my my fuel 
I past to very mine be appear 
April not false my runs harmful cruel 
An that if to the very tear 
I then thy for I hammered muse 
In A to that the physicians more 
Thou holds with parts man so use 
Yet than bitterness high take am before 
Fall of I and poor to foes 
Sin his one evil title doth down 
Thy up are yet love rain those 
Common since be best his be frown 
To bad which mine nothing to all 
Am do self with dwell A call 

2
Seasons scar